In [5]:
from model import *
from data import *
from metrics import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras import backend as K
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_gen_args = dict(rotation_range=30,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=[0.8,1],
                    brightness_range=[0.9,1.1],
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(16,'dataset','images_satellite','labels_satellite',data_gen_args)        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [6]:
model = satellite_unet()
model.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

In [7]:
model_checkpoint = ModelCheckpoint('sat_unet_final.hdf5', monitor='val_mean_iou_weighted',verbose=1, save_best_only=True, mode = 'max')
history=model.fit_generator(train_generator,steps_per_epoch=400,epochs=15,validation_data=val_generator,validation_steps=50, callbacks=[model_checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
400/400 [==============================] - ETA: 0s - loss: 0.0167 - accuracy: 0.6732 - mean_iou: 0.4972 - mean_iou_neg: 0.6241 - mean_iou_weighted: 0.5987 - dice: 0.6289 - dice_neg: 0.7383 - dice_weighted: 0.7164
Epoch 00001: val_mean_iou_weighted improved from -inf to 0.59139, saving model to sat_unet_final.hdf5
400/400 [==============================] - 310s 775ms/step - loss: 0.0167 - accuracy: 0.6732 - mean_iou: 0.4972 - mean_iou_neg: 0.6241 - mean_iou_weighted: 0.5987 - dice: 0.6289 - dice_neg: 0.7383 - dice_weighted: 0.7164 - val_loss: 0.0162 - val_accuracy: 0.6846 - val_mean_iou: 0.4688 - val_mean_iou_neg: 0.6220 - val_mean_iou_weighted: 0.5914 - val_dice: 0.5961 - val_dice_neg: 0.7317 - val_dice_weighted: 0.7046
Epoch 2/15
400/400 [==============================] - ETA: 0s - loss: 0.0098 - accuracy: 0.7117 - mean_iou: 0.5979 - mean_iou_neg: 0.6999 - mean_iou_weighted: 0.6795 - dice: 0.7209 - 

Epoch 12/15
400/400 [==============================] - ETA: 0s - loss: 0.0044 - accuracy: 0.7531 - mean_iou: 0.7431 - mean_iou_neg: 0.8064 - mean_iou_weighted: 0.7937 - dice: 0.8401 - dice_neg: 0.8830 - dice_weighted: 0.8744
Epoch 00012: val_mean_iou_weighted did not improve from 0.65115
400/400 [==============================] - 304s 761ms/step - loss: 0.0044 - accuracy: 0.7531 - mean_iou: 0.7431 - mean_iou_neg: 0.8064 - mean_iou_weighted: 0.7937 - dice: 0.8401 - dice_neg: 0.8830 - dice_weighted: 0.8744 - val_loss: 0.0170 - val_accuracy: 0.6588 - val_mean_iou: 0.5259 - val_mean_iou_neg: 0.6298 - val_mean_iou_weighted: 0.6090 - val_dice: 0.6568 - val_dice_neg: 0.7443 - val_dice_weighted: 0.7268
Epoch 13/15
400/400 [==============================] - ETA: 0s - loss: 0.0043 - accuracy: 0.7524 - mean_iou: 0.7440 - mean_iou_neg: 0.8071 - mean_iou_weighted: 0.7944 - dice: 0.8402 - dice_neg: 0.8832 - dice_weighted: 0.8746
Epoch 00013: val_mean_iou_weighted did not improve from 0.65115
400/400

In [9]:
save_obj(history.history,'satellite_history')

In [9]:
test_img, names = test('dataset/test_images')

folder = './dataset/test_labels'
num_img, size = test_img.shape[0], test_img.shape[1]            

test_label = generate_labels(folder, names, num_img, size)

In [9]:
model_load = load_model('sat_unet_final.hdf5', compile = False, custom_objects={'binary_focal_loss_fixed':binary_focal_loss()})
model_load.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

score, accuracy,  iou, iou_neg, iou_weighted, dice_coeff, dice_coeff_neg, dice_coeff_weighted= model_load.evaluate(test_img, test_label)

1/1 [==============================] - 0s 4ms/step - loss: 0.0170 - accuracy: 0.8709 - mean_iou: 0.4002 - mean_iou_neg: 0.6148 - mean_iou_weighted: 0.5719 - dice: 0.5230 - dice_neg: 0.7128 - dice_weighted: 0.6749


In [125]:
data_gen_args = dict(rotation_range=30,
                    horizontal_flip=True,
                    vertical_flip=True)


test_gen = trainGenerator_simple(4,'dataset','test_images','test_labels', aug_dict = data_gen_args, image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 1,save_to_dir = None,
                   target_size = (256,256),seed = 1)

In [126]:
model = load_model('sat_unet_final.hdf5',  compile = False, custom_objects={'binary_focal_loss_fixed':binary_focal_loss()})
model.compile(optimizer = Adam(lr = 1e-4), loss = [binary_focal_loss()], metrics = ['accuracy', mean_iou, mean_iou_neg, mean_iou_weighted, dice, dice_neg, dice_weighted])

score, accuracy,  iou, iou_neg, iou_weighted, dice_coeff, dice_coeff_neg, dice_coeff_weighted= model.evaluate(test_gen,steps=20)

Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.
20/20 [==============================] - 2s 96ms/step - loss: 0.0287 - accuracy: 0.8596 - mean_iou: 0.3401 - mean_iou_neg: 0.5864 - mean_iou_weighted: 0.5372 - dice: 0.4741 - dice_neg: 0.6903 - dice_weighted: 0.6471
